In [21]:
import asyncio
import requests
import csv

In [2]:
# Retrieve and save all information about a sample of completed mythic runs.
async def retrieve_data(pages_to_sample):
    leaderboard_base_uri = "https://raider.io/api/v1/mythic-plus/runs"
    season = "season-df-2"
    region = "world"
    dungeon = "all"
    affixes = "all"

    rankings = []
    for i in range(pages_to_sample):
        # Requests are limited to 300 per minute, so delay each loop slightly.
        sleep_task = asyncio.sleep(0.5)
        params = {
            "season": season,
            "region": region,
            "dungeon": dungeon,
            "affixes": affixes,
            "page": i
        }
        data = requests.get(leaderboard_base_uri, params=params).json()
        flattened_data = [flatten_data(i) for i in data["rankings"]]
        rankings.extend(flattened_data)
        await sleep_task
    return rankings

# Flatten Data
def flatten_data(ranking):
    flattened_ranking = {
        "score": ranking["score"],
        "dungeon_name": ranking["run"]["dungeon"]["name"],
        "dungeon_num_bosses": ranking["run"]["dungeon"]["num_bosses"],
        "mythic_level": ranking["run"]["mythic_level"],
        "clear_time_ms": ranking["run"]["clear_time_ms"],
        "keystone_time_ms": ranking["run"]["keystone_time_ms"],
        "weekly_modifiers": [i["name"] for i in ranking["run"]["weekly_modifiers"]]
    }
    
    dps_count = 1
    for i in ranking["run"]["roster"]:
        role = i["role"]
        charclass = i["character"]["class"]["name"]
        race = i["character"]["race"]["name"]
        level = i["character"]["level"]
        spec = i["character"]["spec"]["name"]
        if role == "tank":
            tank_details = {
                "tank_class": charclass,
                "tank_race": race,
                "tank_level": level,
                "tank_spec": spec
            }
            flattened_ranking.update(tank_details)
        if role == "healer":
            healer_details = {
                "healer_class": charclass,
                "healer_race": race,
                "healer_level": level,
                "healer_spec": spec
            }
            flattened_ranking.update(healer_details)
        if role == "dps":
            dps_details = {
                f"dps_{dps_count}_class": charclass,
                f"dps_{dps_count}_race": race,
                f"dps_{dps_count}_level": level,
                f"dps_{dps_count}_spec": spec
            }
            dps_count += 1
            flattened_ranking.update(dps_details)
    return flattened_ranking

def write_data(data, filename):
    headers = list(data[0].keys())
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=headers)
        writer.writeheader()
        writer.writerows(data)

Exception ignored in: <finalize object at 0x17490e02e00; dead>
Traceback (most recent call last):
  File "c:\Users\York\AppData\Local\Programs\Python\Python39\lib\weakref.py", line 580, in __call__
    return info.func(*info.args, **(info.kwargs or {}))
  File "c:\Users\York\AppData\Local\Programs\Python\Python39\lib\site-packages\urllib3\connectionpool.py", line 1130, in _close_pool_connections
    conn.close()
  File "c:\Users\York\AppData\Local\Programs\Python\Python39\lib\http\client.py", line 935, in close
    sock.close()   # close it manually... there may be other refs
  File "c:\Users\York\AppData\Local\Programs\Python\Python39\lib\socket.py", line 501, in close
    self._real_close()
  File "c:\Users\York\AppData\Local\Programs\Python\Python39\lib\ssl.py", line 1300, in _real_close
    super()._real_close()
  File "c:\Users\York\AppData\Local\Programs\Python\Python39\lib\socket.py", line 495, in _real_close
    _ss.close(self)
KeyboardInterrupt: 


In [20]:
pages = 100
filename = f"data/rankings{pages}.csv"
raw_rankings = retrieve_data(pages)
rankings = [flatten_data(i) for i in raw_rankings]
write_data(rankings, filename)



You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.
